### Library Imports

In [2]:
import os
import math
import pandas as pd
import torch
from models import Generator
import torchvision.transforms as transforms
from torchvision.transforms.functional import to_tensor
from PIL import Image


## Inference using Generator model

In [4]:
## Set the Device
DEVICE = "cpu"

## Initialize the Generator model
model = Generator(upscale_factor=4).to(DEVICE)

## Load the model weights
state_dict = torch.load('netG_4x_epoch2.pth.tar', map_location=torch.device(DEVICE))
model.load_state_dict(state_dict["model"])

## Set the model to evaluation mode
model.eval()

Generator(
  (initial): ConvBlock(
    (cnn): SeperableConv2d(
      (depthwise): Conv2d(3, 3, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), groups=3)
      (pointwise): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (bn): Identity()
    (act): PReLU(num_parameters=64)
  )
  (residual): Sequential(
    (0): ResidualBlock(
      (block1): ConvBlock(
        (cnn): SeperableConv2d(
          (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): PReLU(num_parameters=64)
      )
      (block2): ConvBlock(
        (cnn): SeperableConv2d(
          (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=Fa

In [11]:
## Load an image
original_hr_image = Image.open('../data/input.png').convert('RGB')

## Create the LR image transformer by downsampling the HR image and applying bicubic interpolation
low_res_image_transform = transforms.Resize((256,256), interpolation=Image.BICUBIC)

## Create HR and LR image scalars
original_hr_image_scalar = transforms.Resize((1024,1024))
original_lr_image_scalar = transforms.Resize((256,256))

## Transform HR image to LR Image
low_res_image = low_res_image_transform(original_hr_image)

## If the input image 
print(original_hr_image.size)

if original_hr_image.size[0] != 1024 or original_hr_image.size[1] != 1024:
    print("[INFO] Resizing the original input HR image to 1024x1024")
    original_hr_image = original_hr_image_scalar(original_hr_image)

# Move the image and model to GPU if available
if DEVICE == "cuda":
    low_res_image = low_res_image.cuda()
    
## Transform the image to tensor
low_res_image = to_tensor(low_res_image)
## Add a batch dimension
low_res_image = low_res_image.unsqueeze(0)

# Perform model inference
with torch.no_grad():
    output_sr_image = model(low_res_image)

## Remove the batch dimension
output_sr_image = output_sr_image.squeeze(0)

## Transform the tensor to PIL image
display_transform = transforms.Compose([transforms.ToPILImage()])
output_sr_image = display_transform(output_sr_image)

## Save the output image
output_sr_image.save("../data/results/output.png")

(1024, 1024)
